In [1]:
import json
import random
from statistics import mean
import numpy as np
import sys
import os
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold
from sklearn.compose import TransformedTargetRegressor
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin
import matplotlib.pyplot as plt
from itertools import accumulate
import pickle
from pathlib import Path
import datetime
from tqdm import tqdm
import torch
import glob

c:\Users\chris\anaconda3\envs\codenames\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        outputs = torch.sigmoid(self.linear(x))
        return outputs

In [3]:
#Load torch model and convert it to sklearn LinearRegression Model
loadname = r'data\score_torch_colt_model.pt'
torch_model = torch.load(loadname)
print('Loaded torch model: ', loadname)

Loaded torch model:  data\score_torch_model-Linear-Final-36-1-35v-2022-11-04-09-41-23.pt


In [6]:
#Create scikit-learn model
skl_model = LinearRegression(fit_intercept=False)
skl_model.coef_ = np.zeros(36)
skl_model.intercept_ = 0
#Cycle through all the coefficients from the torch model

for i in range(36):
    x = torch.zeros(36)
    x[i] = 1.0
    w = torch_model.linear(x).item()
    skl_model.coef_[i] = w
    print(f'Torch weight {i} = {skl_model.coef_[i]}')

#That should do it

Torch weight 0 = -4.695091724395752
Torch weight 1 = -1.8543732166290283
Torch weight 2 = -9.739892959594727
Torch weight 3 = 1.7059537172317505
Torch weight 4 = -1.6369715929031372
Torch weight 5 = 0.00658305361866951
Torch weight 6 = -5.551140308380127
Torch weight 7 = 1.941271185874939
Torch weight 8 = -0.4041709303855896
Torch weight 9 = 0.8298859596252441
Torch weight 10 = -4.566517353057861
Torch weight 11 = 2.274085521697998
Torch weight 12 = 0.49170276522636414
Torch weight 13 = 1.4675524234771729
Torch weight 14 = -3.797506093978882
Torch weight 15 = 2.7123939990997314
Torch weight 16 = 1.1092449426651
Torch weight 17 = 1.9446426630020142
Torch weight 18 = -2.891756772994995
Torch weight 19 = 3.0217623710632324
Torch weight 20 = 1.6080971956253052
Torch weight 21 = 1.9599496126174927
Torch weight 22 = -2.732170820236206
Torch weight 23 = 2.9600441455841064
Torch weight 24 = 1.7915613651275635
Torch weight 25 = 2.1286046504974365
Torch weight 26 = -2.573078155517578
Torch weigh

In [7]:
#Let's verify that things work the same - SANITY CHECK
not_close = 0
N = 1000
for i in range(N):
    x = torch.rand(36)
    ty = torch_model.linear(x).item()
    xn = x.numpy().reshape(1,-1)
    sy = skl_model.predict(xn)
    if not np.isclose(ty,sy):
        print('NOT CLOSE: ty=', ty, ' and sy=', sy)#, 'input was X=', x)
        not_close += 1

print(f'Checked {N} values. {not_close} were not close.')

NOT CLOSE: ty= -0.11406326293945312  and sy= [-0.11406175]
NOT CLOSE: ty= -0.044322967529296875  and sy= [-0.04432438]
NOT CLOSE: ty= 0.017177581787109375  and sy= [0.01717724]
NOT CLOSE: ty= 0.058228492736816406  and sy= [0.05822739]
NOT CLOSE: ty= -0.16495037078857422  and sy= [-0.16494814]
NOT CLOSE: ty= 0.07100105285644531  and sy= [0.07099969]
NOT CLOSE: ty= 0.09995651245117188  and sy= [0.09995489]
NOT CLOSE: ty= -0.029485702514648438  and sy= [-0.02948513]
Checked 1000 values. 8 were not close.


In [8]:
#Save out the sk_learn model, using joblib
save_name = r'data\sklinear_colt_model.joblib'
joblib.dump(skl_model, save_name)

['data\\sklinear36model-nobias-11-04-2022.joblib']